# Regresión Lineal Simple. Ejemplo minimalista

### Importar las librerías relevantes

In [265]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Para graficar en 3-D

### Generar datos al azar para entrenar al modelo

Trabajaremos con dos variables de entrada, las x1 y x2 en nuestros ejemplos anteriores. Se generan al azar a partir de una distribución uniforme.

Se creará una matriz con estas dos variables.  La matriz X del modelo lineal y = x * w + b

In [266]:
# Por facilidad, declaramos una variable que indique el tamaño del conjunto 
#      de datos de entrenamiento.
observaciones = 1000

x1 = np.random.uniform(low=-10, high=10, size=(observaciones,1))
x2 = np.random.uniform(-10, 10, (observaciones,1))

X = np.column_stack((x1,x2))

# Verificar la forma de la matriz 
# Debiera ser n x k, donde n es el número de observaciones, y k es el número de variables.
print (X.shape)

(1000, 2)


### Generar las metas a las que debemos apuntar

Inventaremos una función f(x1, x2) = 2 * x1 - 3 * x2 + 5 + <ruido pequeño>.  El ruido es para hacerlo más realista.

Utilizaremos la metodología de ML, y veremos si el algoritmo la ha aprendido.  

In [267]:
ruido = np.random.uniform(-1, 1, (observaciones,1))

# metas = 2 * x1 - 3 * x2 + 5 + ruido
metas = 13 * x1 + 7 * x2 - 12

# Veamos las dimensiones. Deben ser n x m, donde m es el número de variables de salida.
print (metas.shape)

(1000, 1)


### Graficar los datos a usar para el entrenamiento

La idea es ver que haya una fuerte tendencia que nuestro modelo debe aprender a reproducir.


In [268]:
print(x1.shape)
print(x2.shape)
print(metas.shape)

(1000, 1)
(1000, 1)
(1000, 1)


In [269]:
x1N = x1.reshape(observaciones,)
x2N = x2.reshape(observaciones,)
metasN = metas.reshape(observaciones,)

fig = px.scatter_3d(x = x1N, y = x2N, z = metasN)

fig.update_layout(
    width = 500,
    height = 500,)

fig.show()

### Inicializar variables

Inicializaremos los pesos y sesgos, al azar, dentro de un rango inicial pequeño.  Es posible "jugar" con este valor pero no es recomendable ya que el uso de rangos iniciales altos inhibe el aprendizaje por parte del algoritmo

Los pesos son de dimensiones k x m, donde k es el numero de variables de entrada y m es el número de variables de salida.  

Como solo hay una salida, el sesgo es de tamaño 1, y es un escalar

In [270]:
rango_inicial = 0.1

pesos = np.random.uniform(low = -rango_inicial, high = rango_inicial, size=(2, 1))

sesgos = np.random.uniform(low = -rango_inicial, high = rango_inicial, size=1)

#Veamos cómo fueron inicializados.
print (pesos)
print (sesgos)

[[-0.03685609]
 [ 0.02331572]]
[-0.0153484]


In [271]:
pesos.shape

(2, 1)

### Asignar la tasa de aprendizaje (Eta)

Se asigna un a tasa de aprendizaje pequeña.  Para este ejemplo funciona bien 0.02.  Vale la pena "jugar" con este valor para ver los resultados de hacerlo.

In [272]:
eta = 0.01

### Entrenar el modelo

Usaremos un valor de 100 para iterar sobre el conjunto de datos de entrenamiento.  Ese valor funciona bastante bien con la tasa de aprendizaje de 0.02.  Cómo saber el número adecuado de iteraciones es algo que veremos en futuras sesiones, pero generalmente una tasa de aprendizaje baja requiere de más iteraciones que una más alta.  Sin embargo hay que tener en mente que una tasa de aprendizaje alta puede causar que la pérdida "Loss" diverja a infinito, en vez de converger a cero (0)

Usaremos la función de pérdida L2-norm, pero dividido por 2, para ser consistente con la clase.  Es más, también lo dividiremos por el número de observaciones para obtener un promedio de pérdida por observación.  Hablamos en clase sobre la posiilidad de modificar esta función una vez no se pierda la característica de ser más baja para los resultados mejores, y vice versa.

Imprimimos la función de pérdida (loss) en cada iteración, para ver si está decreciendo como se desea.

Otro pequeño truco es escalar las deltas de la misma manera que se hizo con la función de pérdida.  De esta forma la tasa de aprendizaje es independiente del número de muestras (samples u observaciones).  De nuevo esto no cambia el principio, solo hace más fácil la selección de una tasa única de aprendizaje. 

Finalmente aplicamos la regla de actualización del decenso de gradiente.

Ojo!  los pesos son 2x1, la tasa de aprendizaje es 1x1 (escalar), las entradas son 1000x2, y las deltas escaladas son 1000x1.  Necesitamos obtener la transpuesta de las entradas para que no hayan problemas de dimensión en las operaciones. 



In [273]:
for i in range (100):
    
    # Esta es la ecuacion del modelo lineal: y = xw + b 
    y = np.dot(X, pesos) + sesgos
    
    # Las deltas son las diferencias entre las salidas y las metas (targets)
    # deltas es un vector 1000 x 1
    deltas = y - metas
        
    # perdida = np.sum(deltas ** 2) / 2 / observaciones
    # perdida = np.sum(deltas ** 2) / observaciones

    # Cambio en la función de pérdida L1
    perdida = np.sum(np.abs(deltas)) / observaciones

    
    print(perdida)
    
    deltas_escaladas = deltas / observaciones
      
    pesos = pesos - eta * np.dot(X.T, deltas_escaladas)
    sesgos = sesgos - eta * np.sum(deltas_escaladas)
    
    # Los pesos son actualizados en una forma de algebra lineal(una matriz menos otra)
    # Sin embargo, los sesgos en este caso son solo un número (solo estamos calculando una salida), 
    #       es necesario transformar las deltas a un escalar.      
    # Ambas lineas son consistentes con la metodología de decenso de gradiente-

71.8498983873135
48.01634119552504
32.533354486228156
22.596705269737686
16.38885745845554
12.861483842749296
11.338581740180526
11.060019396507272
10.989726359182146
10.907419376219382
10.817264706336045
10.722100246758119
10.62385764634971
10.523850421328163
10.422970566055309
10.321822691774836
10.220815524571803
10.120224305076661
10.02023333565476
9.920964985843934
9.822499462540483
9.72488828308246
9.62816345539733
9.532343732025378
9.437438869987806
9.343452531853782
9.250384261073659
9.15823082671475
9.066987138703734
8.976646870586054
8.88720288313229
8.798647512358169
8.710972765247206
8.624170452652082
8.53823227944304
8.453149905564526
8.368914987299107
8.285519205067287
8.202954282070117
8.121211996705146
8.040284190749626
7.960162774667372
7.88083973096214
7.802307116204964
7.724557062162448
7.647581776316168
7.571373541970602
7.495924718083785
7.421227738911841
7.347275113529438
7.274059425268237
7.201573331101993
7.1298095609976935
7.058760917245923
6.988420273779384
6.

### Desplegamos los pesos y el sesgo para ver si funcionaron correctamente.

Por el diseño de nuestro datos, los pesos debieran ser 2 y -3, y el sesgo: 5

**NOTA:**  Si aún no están los valores correctos, puede que aún estén convergiendo y sea necesario iterar más veces.  Para esto solo se requiere ejecutar la celda anterior cuantas veces sea requerido

In [274]:
print(pesos, sesgos)      

[[12.96358562]
 [ 7.02953337]] [-7.57588814]


### Graficar las últimas salidas vrs las metas (targets)

Como son las últimas, luego del entrenamiento, representan el modelo final de exactitud.  Entre más cercana esté esta gráfica a una línea de 45 grados, más cercanas están las salidas y metas.

Como este ejemplo es pequeño, es posible hacerlo, en los problemas que veremos más tarde en la clase, esto ya no sería posible.

In [275]:
yN = y.reshape(observaciones,)
metasN = metas.reshape(observaciones,)
fig = px.scatter(x = yN, y =  metasN)

fig.update_layout(
    width = 400,
    height = 400,)

fig.show()